In [20]:
import io 
import numpy as np 
import random

import tensorflow as tf
from tensorflow import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import re

In [38]:
print('Opening file...')
path = "divina_commedia.txt"
with io.open(path, 'r', encoding='utf-8') as file:
    text = file.read().lower()

print("text length", len(text))
print()
print('\n\n\n\n\n', '***** first 1000 characters *****', '\n\n\n\n\n')
text[0:1000]

Opening file...
text length 558240






 ***** first 1000 characters ***** 







"inferno\n\n\n\ninferno: canto i\n\n\nnel mezzo del cammin di nostra vita\n  mi ritrovai per una selva oscura\n  che' la diritta via era smarrita.\n\nahi quanto a dir qual era e` cosa dura\n  esta selva selvaggia e aspra e forte\n  che nel pensier rinova la paura!\n\ntant'e` amara che poco e` piu` morte;\n  ma per trattar del ben ch'i' vi trovai,\n  diro` de l'altre cose ch'i' v'ho scorte.\n\nio non so ben ridir com'i' v'intrai,\n  tant'era pien di sonno a quel punto\n  che la verace via abbandonai.\n\nma poi ch'i' fui al pie` d'un colle giunto,\n  la` dove terminava quella valle\n  che m'avea di paura il cor compunto,\n\nguardai in alto, e vidi le sue spalle\n  vestite gia` de' raggi del pianeta\n  che mena dritto altrui per ogne calle.\n\nallor fu la paura un poco queta\n  che nel lago del cor m'era durata\n  la notte ch'i' passai con tanta pieta.\n\ne come quei che con lena affannata\n  uscito fuor del pelago a la riva\n  si volge a l'acqua perigliosa e guata,\n\ncosi` l'animo mio, 

In [39]:
# Pulizia del testo per rimuovere righe vuote e spazi extra
# Sostituiamo le righe vuote con uno spazio singolo e rimuoviamo gli spazi in eccesso
text = re.sub(r'\n+', '\n', text)  # Rimuove sequenze multiple di linee vuote
text = text.strip()  # Rimuove spazi extra all'inizio e alla fine

# Stampa la lunghezza del testo e i primi 1000 caratteri per il controllo
print("text length", len(text))
print()
print('***** first 1000 characters *****')
print(text[:1000])

# Utilizzare una regex per dividere il testo in base ai canti
# La regex cerca le intestazioni tipo "inferno: canto i", "purgatorio: canto i", "paradiso: canto i" 
# e divide il testo in canti separati

# Regex che cerca i titoli del canto
canti = re.split(r'(?<=inferno|purgatorio|paradiso):\s*canto\s+[ivxlcdm]+', text)

# Rimuoviamo gli spazi extra e gli elementi vuoti dalla lista risultante
canti = [canto.strip() for canto in canti if canto.strip()]

# Verifica la separazione stampando i primi 3 canti
for i in range(min(3, len(canti))):
    print(f"\nCanto {i+1}:")
    print(canti[i][:500])  # Mostra i primi 500 caratteri di ciascun canto
    print("-" * 50)


Canto 1:
inferno
--------------------------------------------------

Canto 2:
inferno
--------------------------------------------------

Canto 3:
nel mezzo del cammin di nostra vita
  mi ritrovai per una selva oscura
  che' la diritta via era smarrita.
ahi quanto a dir qual era e` cosa dura
  esta selva selvaggia e aspra e forte
  che nel pensier rinova la paura!
tant'e` amara che poco e` piu` morte;
  ma per trattar del ben ch'i' vi trovai,
  diro` de l'altre cose ch'i' v'ho scorte.
io non so ben ridir com'i' v'intrai,
  tant'era pien di sonno a quel punto
  che la verace via abbandonai.
ma poi ch'i' fui al pie` d'un colle giunto,
  la` 
--------------------------------------------------


In [ ]:
with io.open("vita_nova.txt", encoding='utf-8') as file:
    external_text = file.read().lower()

print("'vita nova' text length", len(external_text))
print()
print('\n\n\n\n\n', '***** first 1000 characters *****', '\n\n\n\n\n')
external_text[0:1000]

In [ ]:
chars = sorted(list(set(text) | set(external_text)))
print('total chars: ', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars)) # create first dictionary
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)
print(indices_char)

In [ ]:
maxlen = 30 # chunk length
step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('number of sequences: ', len(sentences))
print(sentences[11])
print(next_chars[11])

In [ ]:
# first point of assignment
from sklearn.model_selection import train_test_split

sentences_train, sentences_val, next_chars_train, next_chars_val = train_test_split(
    sentences, next_chars, test_size=0.2, random_state=42
)

print(f"Training set: {len(sentences_train)}")
print(f"Validation set: {len(sentences_val)}")


In [ ]:
external_sentences = []
external_next_chars = []

for i in range(0, len(external_text) - maxlen, step):
    external_sentences.append(external_text[i: i + maxlen])
    external_next_chars.append(external_text[i + maxlen])

print('number of sequences: ', len(external_sentences))
print(external_sentences[11])
print(external_next_chars[11])

In [ ]:
# encode in one rapresentation
print('generating input and output..')

def one_hot_encoding(sentences, next_chars, maxlen, chars, char_indices): 
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
    y = np.zeros((len(sentences), len(chars)), dtype=bool)

    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

    return x, y

# Training set
x_train, y_train = one_hot_encoding(sentences_train, next_chars_train, maxlen, chars, char_indices)

# Validation set
x_val, y_val = one_hot_encoding(sentences_val, next_chars_val, maxlen, chars, char_indices)

# Test set
x_test, y_test = one_hot_encoding(external_sentences, external_next_chars, maxlen, chars, char_indices)

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

# second point
model.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer, 
    metrics=['accuracy']
)

model.summary()

In [10]:
import sys

def testAfterEpoch(epoch, _):
    print()
    print()
    print('***** Epoch: %d *****' % (epoch+1))
    start_index = random.randint(0, len(text)- maxlen - 1)

    generated = ''
    sentence = text[start_index : start_index + maxlen]
    generated = generated + sentence

    print('***** starting sentence *****') 
    print(sentence)
    print('*****************************')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1
        
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


In [11]:
print_callback = LambdaCallback(on_epoch_end=testAfterEpoch)

In [ ]:
model.fit(x_train, y_train,
          batch_size = 2048, 
          epochs = 20, 
          callbacks = [print_callback],
          validation_data =(x_val, y_val))